In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [2]:
import glob
import json
root_path = '/path/to/data/'
metadata_path = f'{root_path}/metadata.csv'
#all_json = glob.glob(f'{root_path}/**/*.json', recursive=True)
all_json = glob.glob(f'{root_path}/comm_use_subset/**/*.json', recursive=True)
len(all_json)


9315

In [3]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            self.qrmine = self.abstract + '<break>' + self.paper_id + '</break>'
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'

    @staticmethod
    def get_qrmine(self):
    	return self.qrmine

first_row = FileReader(all_json[0])
print(first_row)

02049ffbdb20997055c4def3516af0e6e40dd427: ADAR1 isoforms are adenosine deaminases that edit and destabilize double-stranded RNA reducing its immunostimulatory activities. Mutation of ADAR1 leads to a severe neurodevelopmental and inflammatory... Adenosine deaminase acting on dsRNA (ADAR1) isoforms neutralize the immunostimulatory activity of double-stranded RNA (dsRNA) by converting adenosine to inosine (i.e., A-to-I editing) (George et al., ...


In [31]:
dict_ = {'paper_id': [], 'qrmine': []}
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
with open("/tmp/qrmine.txt", "w") as file_object:
    for idx, entry in enumerate(all_json):
        if idx % (len(all_json) // 10) == 0:
            print(f'Processing index: {idx} of {len(all_json)}')
        content = FileReader(entry)
        # get metadata information
        meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
        # no metadata, skip this paper
        if len(meta_data) == 0 or len(content.abstract) == 0:
            continue
        if "eosinophils" in content.abstract \
            or "leukocytes" in content.abstract \
            or "hematocrit" in content.abstract \
            or "hemoglobin" in content.abstract \
            or "platelets" in content.abstract:
            file_object.write(content.qrmine)

Processing index: 0 of 9315
Processing index: 931 of 9315
Processing index: 1862 of 9315
Processing index: 2793 of 9315
Processing index: 3724 of 9315
Processing index: 4655 of 9315
Processing index: 5586 of 9315
Processing index: 6517 of 9315
Processing index: 7448 of 9315
Processing index: 8379 of 9315
Processing index: 9310 of 9315


## Import QRMine

In [32]:
from qrmine import Content
from qrmine import Network
from qrmine import Qrmine
from qrmine import ReadData
from qrmine import Sentiment
from qrmine import MLQRMine
from qrmine import __version__
print(__version__)

3.5.0


In [33]:

## Read the text file(s)
data = ReadData()
inp_file = ["/tmp/qrmine.txt"]
data.read_file(inp_file)

In [34]:
print(len(data.documents))

73


## Display categories --- recurring concepts in text

In [35]:
import textacy
from random import randint
q = Qrmine()
all_articles = Content(data.content)
doc = textacy.make_spacy_doc(all_articles.doc)
q.print_categories(doc, 10) # 10 categories


---Categories with count---
| CATEGORY     | WEIGHT                |
| cell         | 0.00958045444832253   |
| infection    | 0.005235415627893724  |
| increase     | 0.0031341263622765153 |
| +            | 0.0028492057838877412 |
| response     | 0.0028135907115891447 |
| mouse        | 0.002742360566991951  |
| result       | 0.002742360566991951  |
| leukocyte    | 0.0026711304223947575 |
| inflammatory | 0.002599900277797564  |
| virus        | 0.002564285205498967  |
---------------------------



['cell',
 'infection',
 'increase',
 '+',
 'response',
 'mouse',
 'result',
 'leukocyte',
 'inflammatory',
 'virus']

## Display coding dictionary for qualitative analysis (Grounded Theory)

In [36]:
q.print_dict(all_articles, 5)



---Coding Dictionary---
| CATEGORY | PROPERTY   | DIMENSION    |
| increase | cell       | cellular     |
| ...      | ...        | inflammatory |
| ...      | ...        | include      |
| ...      | expression | increase     |
| ...      | ...        | inflammatory |
| ...      | ...        | follow       |
| ...      | infection  | viral        |
| ...      | ...        | inflammatory |
| ...      | ...        | immune       |
| induce   | cell       | cellular     |
| ...      | ...        | inflammatory |
| ...      | ...        | include      |
| ...      | protein    | viral        |
| ...      | ...        | induce       |
| ...      | ...        | serum        |
| ...      | study      | study        |
| ...      | ...        | present      |
| ...      | ...        | investigate  |
| include  | cell       | cellular     |
| ...      | ...        | inflammatory |
| ...      | ...        | include      |
| ...      | patient    | compare      |
| ...      | ...        | includ

## Display Topics

In [37]:
q.content = data
q.process_content()
q.print_topics(5)


---Topics---
| TOPIC   | DESCRIPTION                                                                                         |
| TOPIC:1 | +   monocyte   inflammatory   cell   virus   macrophage   high   expression   cancer   infiltration |
| TOPIC:2 | il   il num   num   liver   induce   cytokine   immune   type   infection   production              |
| TOPIC:3 | β   enzyme   gene   function   follow   flow   finding   find   fever   factor                      |
---------------------------



In [18]:

q.category_association(10)



---Categories with count---
| CATEGORY                   | WEIGHT               |
| leukocyte                  | 0.05172413793103448  |
| pulmonary                  | 0.034482758620689655 |
| syndrome                   | 0.034482758620689655 |
| sequestration              | 0.034482758620689655 |
| sequestration of leukocyte | 0.034482758620689655 |
| Leukocyte                  | 0.017241379310344827 |
---------------------------


---Categories with count---
| CATEGORY        | WEIGHT               |
| ibalt           | 0.034482758620689655 |
| iBALT           | 0.027586206896551724 |
| pulmonary       | 0.020689655172413793 |
| inflammation    | 0.020689655172413793 |
| area            | 0.013793103448275862 |
| participate     | 0.013793103448275862 |
| immune          | 0.013793103448275862 |
| response        | 0.013793103448275862 |
| immunity        | 0.013793103448275862 |
| immune response | 0.013793103448275862 |
---------------------------


---Categories with count---
| CA

,support,itemsets
